<a href="https://colab.research.google.com/github/dudaspm/IST210/blob/master/StartingSQL/LearningSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is setup to let you learn about SQL. 

First, let's get the data we will be working with today. 

#### Seattle Weather
from: https://vega.github.io/vega-datasets 

In [16]:
#Used to make requests
import urllib

f = urllib.request.urlopen("https://vega.github.io/vega-datasets/data/seattle-weather.csv")
text = str(f.read().decode('utf-8'))

for line in text.split("\n")[0:5]:
  print (line.split(","))

['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather']
['2012/01/01', '0.0', '12.8', '5.0', '4.7', 'drizzle']
['2012/01/02', '10.9', '10.6', '2.8', '4.5', 'rain']
['2012/01/03', '0.8', '11.7', '7.2', '2.3', 'rain']
['2012/01/04', '20.3', '12.2', '5.6', '4.7', 'rain']


Let's create a database and table to house this data. For this exercise, we will be using SQLite. 

In [0]:
import sqlite3
# using the SQLite Table to read data.
con = sqlite3.connect('WeatherData') 

In [0]:
cursor = con.cursor()
cursor.execute('''create TABLE weather(date text PRIMARY KEY, precipitation real, temp_max real, temp_min real, wind real, weather text)''')
con.commit()

In [0]:
cursor = con.cursor()
cursor.execute('''insert INTO weather VALUES ('2012/01/01', 0.0, 12.8, 5.0, 4.7, 'drizzle' );''')
con.commit()

In [25]:
cursor = con.cursor()
cursor.execute('''select * from weather;''')
print (cursor.fetchall())
con.commit()

[('2012/01/01', 0.0, 12.8, 5.0, 4.7, 'drizzle')]
